ROBÔ PARA DOWNLOAD DOS DADOS DA CVM

REGRA DE NEGÓCIO ATUALIZADA:
- Para arquivos históricos, o download é pulado se o arquivo já existir.
- Para o arquivo do MÊS CORRENTE, o download é sempre forçado para garantir
  a versão mais recente, substituindo o arquivo local se ele existir.

In [13]:
import logging
import time
from datetime import datetime
from pathlib import Path
import requests
import zipfile

# --- CONFIGURAÇÃO CENTRAL ---
CONFIG = {
    "PASTA_DOWNLOAD_RAW": Path("01_downloads_raw_zips"),
    "PASTA_DADOS_EXTRAIDOS": Path("02_dados_extraidos_csv"),
    "URL_BASE_ANUAIS": "http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/HIST/",
    "NOME_ARQUIVO_ANUAL": "inf_diario_fi_{ano:04d}.zip",
    "ANO_INICIAL_HISTORICO": 2000,
    "ANO_FINAL_HISTORICO": 2021,
    "URL_BASE_MENSAIS": "http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/",
    "NOME_ARQUIVO_MENSAL": "inf_diario_fi_{ano:04d}{mes:02d}.zip",
    "ANO_INICIAL_MENSAL": 2021,
    "PAUSA_REQUISICOES_SEG": 1.0,
    "USER_AGENT": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
}

# --- Configuração do Logging ---
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)


def criar_pastas_necessarias():
    logging.info("Verificando e criando pastas de trabalho...")
    CONFIG["PASTA_DOWNLOAD_RAW"].mkdir(parents=True, exist_ok=True)
    CONFIG["PASTA_DADOS_EXTRAIDOS"].mkdir(parents=True, exist_ok=True)


def baixar_arquivo(session: requests.Session, url: str, caminho_salvar: Path) -> bool:
    if caminho_salvar.exists():
        logging.info(f"Arquivo já existe, pulando: {caminho_salvar.name}")
        return True
    try:
        logging.info(f"Baixando: {url}")
        with session.get(url, stream=True, timeout=60) as r:
            if r.status_code == 404:
                logging.warning(f"Arquivo não encontrado (404): {url}")
                return False
            r.raise_for_status()
            with open(caminho_salvar, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
            logging.info(f"Salvo com sucesso: {caminho_salvar.name}")
            return True
    except requests.exceptions.RequestException as e:
        logging.error(f"Falha ao baixar {url}: {e}")
        return False


def baixar_dados_anuais(session: requests.Session):
    logging.info("--- INICIANDO DOWNLOAD DOS DADOS ANUAIS HISTÓRICOS ---")
    ano_inicial = CONFIG["ANO_INICIAL_HISTORICO"]
    ano_final = CONFIG["ANO_FINAL_HISTORICO"]
    for ano in range(ano_final, ano_inicial - 1, -1):
        nome_arquivo = CONFIG["NOME_ARQUIVO_ANUAL"].format(ano=ano)
        url_completa = CONFIG["URL_BASE_ANUAIS"] + nome_arquivo
        caminho_salvar = CONFIG["PASTA_DOWNLOAD_RAW"] / nome_arquivo
        baixar_arquivo(session, url_completa, caminho_salvar)
        time.sleep(CONFIG["PAUSA_REQUISICOES_SEG"])


# <<< INÍCIO DA ALTERAÇÃO >>>
def baixar_dados_mensais(session: requests.Session):
    """
    Baixa os arquivos mensais.
    Força o download para o mês corrente (M) e o mês anterior (M-1),
    removendo os arquivos locais se eles existirem.
    """
    logging.info("--- INICIANDO DOWNLOAD DOS DADOS MENSAIS RECENTES ---")
    try:
        from dateutil.relativedelta import relativedelta
    except ImportError:
        logging.error("Biblioteca 'python-dateutil' não encontrada. Instale com: pip install python-dateutil")
        return
        
    hoje = datetime.now()
    mes_anterior = hoje - relativedelta(months=1) # Calcula o mês anterior
    data_inicio_busca = datetime(CONFIG["ANO_INICIAL_MENSAL"], 1, 1)
    
    data_corrente_loop = hoje
    while data_corrente_loop >= data_inicio_busca:
        nome_arquivo = CONFIG["NOME_ARQUIVO_MENSAL"].format(ano=data_corrente_loop.year, mes=data_corrente_loop.month)
        caminho_salvar = CONFIG["PASTA_DOWNLOAD_RAW"] / nome_arquivo
        url_completa = CONFIG["URL_BASE_MENSAIS"] + nome_arquivo

        # Verifica se o arquivo em questão é do mês corrente (M)
        eh_mes_corrente = (data_corrente_loop.year == hoje.year and 
                           data_corrente_loop.month == hoje.month)
        
        # Verifica se o arquivo em questão é do mês anterior (M-1)
        eh_mes_anterior = (data_corrente_loop.year == mes_anterior.year and 
                           data_corrente_loop.month == mes_anterior.month)

        # Se for M ou M-1 e o arquivo já existir, remove para forçar o download
        if (eh_mes_corrente or eh_mes_anterior) and caminho_salvar.exists():
            logging.info(f"Arquivo de atualização frequente (M ou M-1). Removendo versão antiga: {caminho_salvar.name}")
            caminho_salvar.unlink()

        baixar_arquivo(session, url_completa, caminho_salvar)
        
        data_corrente_loop -= relativedelta(months=1)
        time.sleep(CONFIG["PAUSA_REQUISICOES_SEG"])
# <<< FIM DA ALTERAÇÃO >>>


def extrair_arquivos_zip():
    logging.info("--- INICIANDO EXTRAÇÃO DOS ARQUIVOS .ZIP ---")
    pasta_raw = CONFIG["PASTA_DOWNLOAD_RAW"]
    pasta_extraidos = CONFIG["PASTA_DADOS_EXTRAIDOS"]
    logging.info(f"Limpando pasta de destino antes da extração: '{pasta_extraidos}'")
    for arquivo_antigo in pasta_extraidos.glob("*"):
        try:
            if arquivo_antigo.is_file():
                arquivo_antigo.unlink()
        except Exception as e:
            logging.error(f"Não foi possível remover o arquivo antigo {arquivo_antigo.name}: {e}")
    arquivos_zip = list(pasta_raw.glob("*.zip"))
    if not arquivos_zip:
        logging.warning("Nenhum arquivo .zip encontrado na pasta de downloads para extrair.")
        return
    logging.info(f"Encontrados {len(arquivos_zip)} arquivos .zip para processar.")
    for caminho_zip in arquivos_zip:
        logging.info(f"Extraindo: {caminho_zip.name} -> para '{pasta_extraidos}'")
        try:
            with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
                zip_ref.extractall(pasta_extraidos)
        except zipfile.BadZipFile:
            logging.error(f"Erro: O arquivo {caminho_zip.name} parece estar corrompido ou não é um .zip válido.")
        except Exception as e:
            logging.error(f"Falha inesperada ao extrair {caminho_zip.name}: {e}")
    logging.info("--- Processo de extração concluído. A pasta de destino foi reconstruída. ---")


def main():
    """Função principal que orquestra todo o processo."""
    logging.info(">>> INICIANDO ROTINA DE CARGA TOTAL DE DADOS DE FUNDOS CVM <<<")
    criar_pastas_necessarias()
    headers = {'User-Agent': CONFIG['USER_AGENT']}
    with requests.Session() as session:
        session.headers.update(headers)
        baixar_dados_anuais(session)
        baixar_dados_mensais(session)
    extrair_arquivos_zip()
    logging.info(">>> ROTINA FINALIZADA COM SUCESSO! <<<")


if __name__ == "__main__":
    main()

2025-08-03 19:51:20 [INFO] - >>> INICIANDO ROTINA DE CARGA TOTAL DE DADOS DE FUNDOS CVM <<<
2025-08-03 19:51:20 [INFO] - Verificando e criando pastas de trabalho...
2025-08-03 19:51:20 [INFO] - --- INICIANDO DOWNLOAD DOS DADOS ANUAIS HISTÓRICOS ---
2025-08-03 19:51:20 [INFO] - Baixando: http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/HIST/inf_diario_fi_2021.zip
2025-08-03 19:51:20 [WARNING] - Arquivo não encontrado (404): http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/HIST/inf_diario_fi_2021.zip
2025-08-03 19:51:21 [INFO] - Arquivo já existe, pulando: inf_diario_fi_2020.zip
2025-08-03 19:51:22 [INFO] - Arquivo já existe, pulando: inf_diario_fi_2019.zip
2025-08-03 19:51:23 [INFO] - Arquivo já existe, pulando: inf_diario_fi_2018.zip
2025-08-03 19:51:24 [INFO] - Arquivo já existe, pulando: inf_diario_fi_2017.zip
2025-08-03 19:51:25 [INFO] - Arquivo já existe, pulando: inf_diario_fi_2016.zip
2025-08-03 19:51:26 [INFO] - Arquivo já existe, pulando: inf_diario_fi_2015.zip
2025-0